In [1]:
# FIX BUG WITH SQLITE
# https://gist.github.com/defulmere/8b9695e415a44271061cc8e272f3c300

__import__('pysqlite3')
import sys
import os
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join('..', 'db.sqlite3'),
    }
}

In [2]:
import os
import openai
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
from chromadb.utils import embedding_functions
from langchain.schema.document import Document

In [3]:
import pandas as pd

In [5]:
data = pd.read_csv("../src/artifacts/@karpovcourseschat.csv", delimiter=";")

In [6]:
data


,message_id,date,text
0,192405,2024-01-05 11:11:44+00:00,"а я сильно загружал: обучался питону. ролики, ..."
1,192404,2024-01-05 11:05:58+00:00,Не особо
2,192403,2024-01-05 11:02:47+00:00,"видимо, нормально для этих моделей 🤔"
3,192402,2024-01-05 11:02:25+00:00,оперативу загружаешь?
4,192401,2024-01-05 11:00:51+00:00,"У меня такой мак уже год, 90% времени подключе..."
5,192400,2024-01-05 10:59:43+00:00,с августа 21 года пользовались\nсерийник бьётс...
6,192399,2024-01-05 10:59:29+00:00,hdmi
7,192398,2024-01-05 10:58:59+00:00,к монитору через type c или hdmi???\nтут же ес...
8,192397,2024-01-05 10:58:58+00:00,?
9,192396,2024-01-05 10:58:27+00:00,бу сколько


# Создаем функцию эмбеддингов

In [7]:
load_dotenv(dotenv_path='../.env')
PROXY_API_KEY = os.getenv("PROXY_API_KEY")


In [8]:


emb_fn = OpenAIEmbeddings(
    api_key=os.getenv("PROXY_API_KEY"),
    model="text-embedding-ada-002",
    base_url="https://api.proxyapi.ru/openai/v1",
)

In [9]:
emb_fn

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7fe21d189f60>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7fe21d0041c0>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base='https://api.proxyapi.ru/openai/v1', openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='sk-p43vYM3mtCnvdV1VF8Nm8CceANp4uO49', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None)

# Создаем ChromaDB

In [12]:
chroma_client = chromadb.PersistentClient(path="./chroma_db_2")


In [14]:
collection = chroma_client.get_or_create_collection(name="my_collection")

In [16]:
docs = data["text"].to_list()

collection.add(
    documents=docs,
    embeddings=emb_fn.embed_documents(docs),
    metadatas=[{"source": "local"} for _ in docs],
    ids=[f"id{i}" for i in range(len(docs))],
)

In [19]:
collection = chroma_client.get_collection(name = "my_collection")

In [26]:
user_query = "как дела?"

search_result = collection.query(
  query_embeddings=emb_fn.embed_query(user_query),
  n_results=5
)

In [32]:
results = search_result['documents'][0][::-1]

['к монитору через type c или hdmi???\nтут же есть разница большая',
 'бу сколько',
 '?',
 'видимо, нормально для этих моделей 🤔',
 'оперативу загружаешь?']

[Collection(name=my_collection)]